In [1]:
from bert import Ner
import pandas as pd
df = pd.read_csv("./z_processed-data.csv")
model = Ner("out_large/")

In [5]:
from nltk import ToktokTokenizer
from nltk.tokenize import sent_tokenize

def sentenced(x):
  entities = []
  count = 0 
  li = sent_tokenize(x)
  for sentence in li:
    if len(sentence) < 512:
      atomised = model.predict(sentence)
      for atoms in atomised :
        if atoms['tag'] != 'O':
          entities.append(atoms)
  return entities

df["entities"] = df["3"].apply(lambda x: sentenced(str(x)))

In [43]:
df.to_csv('./y_processed_dat.csv')


NameError: name 'df' is not defined

In [2]:
import ast
import pandas as pd
df1 = pd.read_csv("./y_processed_dat.csv")

In [3]:
for k, v in df1["entities"].items():
    a = ast.literal_eval(v)
    name = {'persons': [], 'locations': [], 'orgs': []}
    for atom in a:
        if atom['tag'].startswith('B-PER'):
            name['persons'].append(atom['word'])
        elif atom['tag'].startswith('I-PER'):
            try:
                name['persons'][-1] += " " + atom['word']
            except IndexError:
                name['persons'].append(atom['word'])

        elif atom['tag'].startswith('B-ORG'):
            name['orgs'].append(atom['word'])
        elif atom['tag'].startswith('I-ORG'):
            try:
                name['orgs'][-1] += " " + atom['word']
            except IndexError:
                name['orgs'].append(atom['word'])

        elif atom['tag'].startswith('B-LOC'):
            name['locations'].append(atom['word'])
        elif atom['tag'].startswith('I-LOC'):
            try:
                name['locations'][-1] += " " + atom['word']
            except IndexError:
                name['locations'].append(atom['word'])

    for k1, v1 in name.items():
        df1.at[k,k1]= str(v1)
df1.head()

,Unnamed: 0,Unnamed: 0.1,0,1,2,3,secondary_data,entities,persons,locations,orgs
0,0,0,Adzhubei__Aleksei,March_12__1962,Alexei_Adzhubei-s_Account_of_His_Visit_to_Wash...,"TOP SECRETDuring my visit to Washington, Brazi...",TOP SECRETDuring my visit to Washington Brazil...,"[{'word': 'Washington', 'tag': 'B-LOC', 'confi...","['John Kennedy', 'Robert Kennedy', 'João ] Gou...","['Washington', 'Brazil', 'Mexico', 'US', 'Braz...","['International', 'Central Committee', 'Saling..."
1,1,1,Albania__Ministry_of_Foreign_Affairs,March_15__1971,Notes_on_a_Bulletin_of_the_Korean_News_Agency_0,[Handwritten document] Note Looking at the bul...,Handwritten document Note Looking at the bul...,"[{'word': 'Korean', 'tag': 'B-MISC', 'confiden...",[],[],"['5th', 'Congress of the Korean Workers ’ Party']"
2,2,2,Alexandru__Boaba,April_06__1978,TELEGRAM_075_205_from_the_Romanian_Embassy_in_...,NaN,NaN,[],[],[],[]
3,3,3,Anda__Torleiv__1921-,October_21__1976,Telegram_from_the_Embassy_in_Beijing__-Smuggli...,ROYAL MINISTRY OF FOREIGN AFFAIRSCOPY NO:1: MI...,ROYAL MINISTRY OF FOREIGN AFFAIRSCOPY NO:1: MI...,"[{'word': 'POLITICAL', 'tag': 'B-MISC', 'confi...","['Hyeon Jun-geuk', 'Hyun Jun Guk', 'Torleiv', ...","['BEIJINGNR', 'OCTOBER', 'Oslo', 'North Korea'...","['Ministry of Foreign Affairs', 'Ministry of F..."
4,4,4,Anda__Torleiv__1921-,October_22__1976,Telegram_from_Norwegian_Ambassador_to_China_to...,"22.10.76, 09.34 amCOPY NO:1: MINISTER OF FOREI...",221076 0934 amCOPY NO:1: MINISTER OF FOREIGN A...,"[{'word': 'MINISTER', 'tag': 'B-LOC', 'confide...","['UDCINBOUND', 'Keuk', 'Hyeon Jun-geuk', 'Hyon...","['MINISTER', 'BEIJINGNo', 'NORTH KOREA', 'OSLO...",[]


In [4]:
def clean_date(date_str):
    try:
        date_arr = date_str.split('_')
        middle_token = '-'
        if len(date_arr) >1 and date_arr[1] != '':
            middle_token = '-' + date_arr[1] + '-'
        new = date_arr[0] + middle_token + date_arr[-1]
    except:
        print(date_str)
        new = ""
    return pd.to_datetime(new,errors='coerce')
df1["time"] = df1["1"].apply(clean_date)
df1.sort_values(by=['time'], inplace=True)


In [5]:
def boolean_df(dataframe, org_chart):# Create empty dict
    bool_dict = {}
    
    # Loop through all the tags
    for i, item in org_chart.items():
        #print(item)
        # Apply boolean mask
        dataframe[item] = dataframe["persons"].apply(lambda x: i in x)
            
    # Return the results as a dataframe
    return pd.DataFrame(dataframe)

In [8]:
org_chart = pd.read_csv("./merged_political_military.csv")
org_chart.head()
org_dict = org_chart[['itemLabel', 'politicalPartyLabel']].set_index('itemLabel').to_dict()
df2 = boolean_df(df1, org_dict['politicalPartyLabel'])
df2.to_csv("../persons_involved_in_doc.csv")